# Extract data from a map service

https://support.esri.com/en-us/knowledge-base/how-to-how-to-extract-data-from-a-map-service-using-pyt-000019645

In [1]:
import urllib.parse
import urllib.request 
import os
import arcpy
import json
import pandas as pd

In [2]:
# Add the main filepath
main_file_path = "//192.168.101.45/gis/_Projects/SpeciesDataPrep_20241209_48ad30"

#Setup the Pro environment
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

fgdb = main_file_path + "/SpeciesDataPrep_Python.gdb"
arcpy.env.workspace = fgdb

In [3]:
# Location where data will be downloaded
download_location = main_file_path + "/commondata/userdata/Data_download/Downloads"
item_url_file = "ItemURL_ArcGIS_6monthUpdates.csv"

In [4]:
# Change the current working directory
os.chdir(download_location)

In [5]:
os.getcwd()

'\\\\192.168.101.45\\gis\\_Projects\\SpeciesDataPrep_20241209_48ad30\\commondata\\userdata\\Data_download\\Downloads'

In [6]:
# Read the csv file that contains the desired URL
item_url_df = pd.read_csv(main_file_path + "/commondata/userdata/Data_download/Source_files/" + item_url_file)
item_url_df.head(3)

,Dataset,Owner,URL
0,Local Nature Reserves (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
1,National Nature Reserves (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
2,Priority Habitats Inventory (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...


In [7]:
# Create a list of the item URLs
url_list = item_url_df['URL'].tolist()
url_list[:4]

['https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Local_Nature_Reserves_England/FeatureServer/0/query?',
 'https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/National_Nature_Reserves_England/FeatureServer/0/query?',
 'https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Priority_Habitats_Inventory_England/FeatureServer/0/query?',
 'https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Ramsar_England/FeatureServer/0/query?']

In [8]:
# Create a list of the item names
name_list = item_url_df['Dataset'].tolist()
name_list[:4]

['Local Nature Reserves (England)',
 'National Nature Reserves (England)',
 'Priority Habitats Inventory (England)',
 'Ramsar (England)']

In [9]:
# Query the parameters
params = {'where': '1=1',
          'geometryType': 'esriGeometryEnvelope',
          'spatialRel': 'esriSpatialRelIntersects',
          'relationParam': '',
          'outFields': '*',
          'returnGeometry': 'true',
          'geometryPrecision':'',
          'outSR': '',
          'returnIdsOnly': 'false',
          'returnCountOnly': 'false',
          'orderByFields': '',
          'groupByFieldsForStatistics': '',
          'returnZ': 'false',
          'returnM': 'false',
          'returnDistinctValues': 'false',
          'f': 'pjson'
         }

encode_params = urllib.parse.urlencode(params).encode("utf-8")

In [10]:
# Create a request and read it using urllib
counter = 0
for url in url_list:
    response = urllib.request.urlopen(url, encode_params)
    json = response.read()
    
    # Write the JSON response to text file.
    with open(name_list[counter] + ".json", "wb") as ms_json:
        ms_json.write(json)
    
    # Convert JSON to shapefile using the JSONToFeatures function.
    ws = os.getcwd() + os.sep
    arcpy.JSONToFeatures_conversion(name_list[counter] + ".json", ws + name_list[counter] + ".shp")
    counter += 1

In [ ]:
# magic command to clear all variables in the namespace
# https://community.esri.com/t5/python-documents/arcgis-notebooks-in-arcgis-pro-3-1-faq/ta-p/1261606/jump-to/first-unread-message
%reset -f